In [1]:
import cv2
import imutils
import face_recognition
from deepface import DeepFace
from PIL import Image
import numpy as np
import torch
import pathlib



### set the offset for hair and extended chin detection

In [2]:
offsetPercentageWidth = 10
offsetPercentageHeight = 30

### Load detail extraction model for photo extraction

In [3]:
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

aadhaar_model = torch.hub.load('yolov5', 'custom', path = r'best.pt', force_reload = True, source='local')

YOLOv5  v7.0-280-g4878541d Python-3.10.13 torch-2.2.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7050580 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


# figure out a solution for this where if multiple faces are detected it should return multiple images and if no face detected then the message should be "No face detected"

In [4]:
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
def detect_eye(img):
    eye_img = img.copy()
    eye_rect = eye_cascade.detectMultiScale(eye_img,scaleFactor=1.2,minNeighbors=5)

    print(eye_rect)

    if len(eye_rect) == 1 or len(eye_rect) == 0:

        return cv2.putText(eye_img, 'Ensure both your eyes are open and looking at camera', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_4)
    
    elif len(eye_rect) > 2:
        return cv2.putText(eye_img, "Make sure your there is no one in your background", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_4)
    
    else:
    
        for (x,y,w,h) in eye_rect:
            cv2.rectangle(eye_img,(x,y),(x+w,y+h),(255,255,255),10)
    return eye_img

In [5]:
def get_face_from_image(input_image):

    face_locations = face_recognition.face_locations(input_image)

    if len(face_locations) > 1 :

        print("Multiple faces detected")
        return ["Multiple faces detected"]

    for face_location in face_locations:

        # Print the location of each face in this image
        top, right, bottom, left = face_location
        print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

        # You can access the actual face itself like this:
        # input_image = input_image[top:bottom, left:right]

        return [top, right, bottom, left]

In [22]:
def draw_rectangle(image, top, right, bottom, left, color=(0, 255, 0), thickness=2):
    cv2.rectangle(image, (left, top), (right, bottom), color, thickness)
    return image


cap = cv2.VideoCapture(0)
list_of_frames = []
frame_copy = None

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Resize the frame
    frame = imutils.resize(frame, width=640, height=640)

    frame_copy = frame.copy()

    # Draw a bigger rectangle for the region of interest
    rectangle_width = 320
    rectangle_height = 320
    rect_start_x = (frame.shape[1] - rectangle_width) // 2
    rect_start_y = (frame.shape[0] - rectangle_height) // 2
    rect_end_x = rect_start_x + rectangle_width
    rect_end_y = rect_start_y + rectangle_height
    cv2.rectangle(frame, (rect_start_x, rect_start_y), (rect_end_x, rect_end_y), (255, 0, 0), 2)

    # Get face coordinates if there
    detection_of_face = get_face_from_image(frame)

    if detection_of_face is not None:
        if len(detection_of_face) == 1:
            list_of_frames.clear()
            cv2.putText(frame, 'Multiple faces detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_4)
        else:
            # Check if the face is inside the bigger rectangle
            if len(detection_of_face) == 4 and rect_start_x <= detection_of_face[3] <= rect_end_x and rect_start_y <= detection_of_face[0] <= rect_end_y:
                frame = draw_rectangle(frame, detection_of_face[0], detection_of_face[1], detection_of_face[2], detection_of_face[3], (0, 255, 255), 2)
                frame = detect_eye(frame)

                # store the frame if there is no movement

                if len(list_of_frames) == 0:
                    list_of_frames.append(detection_of_face)

                elif len(list_of_frames) <= 4 and list_of_frames[-1] == detection_of_face:
                    list_of_frames.append(detection_of_face)

                elif len(list_of_frames) == 5:
                    print("image captured for verification")
                    cv2.imwrite("kyc_liveness_detection/captured_image.jpeg", frame)
                    break
                
                else:
                    list_of_frames.clear()


                

            # else:
            #     cv2.putText(frame, 'Place your face inside the rectangle', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_4)
    else:
        list_of_frames.clear()
        cv2.putText(frame, 'Ensure your face is inside the box', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_4)

    # Display the frame
    cv2.imshow('Webcam', frame)

    # Wait for the user to press 'c' to capture an image
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        # Save the captured image (you can change the filename as needed)
        cv2.imwrite('captured_image.jpg', frame)
        print("Image captured!")

    # Break the loop if the user presses 'q'
    elif key == ord('q'):
        break

# Release the webcam and close the OpenCV window
cap.release()
cv2.destroyAllWindows()


Multiple faces detected
Multiple faces detected
Multiple faces detected
Multiple faces detected
Multiple faces detected
Multiple faces detected
Multiple faces detected
A face is located at pixel location Top: 254, Left: 540, Bottom: 383, Right: 640
A face is located at pixel location Top: 196, Left: 196, Bottom: 325, Right: 325
[[578 254  45  45]]
A face is located at pixel location Top: 295, Left: 195, Bottom: 385, Right: 285
[[459 414  54  54]]
A face is located at pixel location Top: 153, Left: 0, Bottom: 282, Right: 124
A face is located at pixel location Top: 170, Left: 508, Bottom: 232, Right: 570
Multiple faces detected
Multiple faces detected
Multiple faces detected
Multiple faces detected
Multiple faces detected
Multiple faces detected
Multiple faces detected
Multiple faces detected
Multiple faces detected
A face is located at pixel location Top: 343, Left: 557, Bottom: 450, Right: 640
A face is located at pixel location Top: 259, Left: 0, Bottom: 367, Right: 103
A face is loc

In [23]:
list_of_frames


[[222, 448, 407, 262],
 [222, 448, 407, 262],
 [222, 448, 407, 262],
 [222, 448, 407, 262],
 [222, 448, 407, 262]]

In [17]:
detections = list_of_frames[-1]

In [19]:
image1 = frame_copy[detections[0]:detections[2], detections[3]:detections[1]]
image1 = Image.fromarray(image1)
image1.show()


In [10]:
image2 = cv2.imread("..\\research\\aadhaar_photo.jpg")
image2 = Image.fromarray(image2)
image2.show()

In [11]:
# Convert images to grayscale
image1_gray = cv2.cvtColor(np.array(image1), cv2.COLOR_BGR2RGB)
image2_gray = cv2.cvtColor(np.array(image2), cv2.COLOR_BGR2RGB)

In [12]:
result = DeepFace.verify(img1_path = np.array(image1_gray), img2_path = np.array(image2_gray), enforce_detection=False)

In [13]:
result

{'verified': False,
 'distance': 0.742799004057275,
 'threshold': 0.68,
 'model': 'VGG-Face',
 'detector_backend': 'opencv',
 'similarity_metric': 'cosine',
 'facial_areas': {'img1': {'x': 0, 'y': 0, 'w': 155, 'h': 155},
  'img2': {'x': 53, 'y': 74, 'w': 252, 'h': 252}},
 'time': 5.19}

In [20]:
# offset trial


top, right, bottom, left = detections

width = right - left
height = bottom - top

offsetWidth = (offsetPercentageWidth/100)*width
offsetHeight = (offsetPercentageHeight/100)*height

left = int(left - offsetWidth)
right = int(right + offsetWidth)
top = int(top - offsetHeight*2)
bottom = int(bottom + offsetHeight)



In [21]:
img = frame_copy[top:bottom, left:right]
img = Image.fromarray(img)
img.show()